In [1]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp, Statevector
import numpy as np

# --- Fake dataset (binary labels)
y_true = np.array([0, 1, 1, 0])  # 4 samples
num_qubits = len(y_true)

# --- Define a cost Hamiltonian
# If label = 0 → want qubit in |0⟩ (Z=+1)
# If label = 1 → want qubit in |1⟩ (Z=-1)
# Cost per qubit = 0.5 * (1 - (-1)^y_true * Z)
pauli_labels = []
coeffs = []

for i, y in enumerate(y_true):
    sign = -1 if y == 1 else 1
    label = ['I'] * num_qubits
    label[i] = 'Z'
    pauli_labels.append(''.join(label))
    coeffs.append(0.5 * (1 - sign))

hamiltonian = SparsePauliOp(pauli_labels, coeffs)

# --- Prepare a classifier candidate as a basis state
# Example classifier predicts: [0, 1, 0, 0] → bitstring = 0100
bitstring = '0100'[::-1]  # little endian
qc = QuantumCircuit(num_qubits)
for i, b in enumerate(bitstring):
    if b == '1':
        qc.x(i)

# --- Simulate the circuit
state = Statevector.from_instruction(qc)

# --- Measure expected classification cost
energy = np.real(state.expectation_value(hamiltonian))
print(f"Classification cost (energy): {energy}")


Classification cost (energy): 0.0
